In [1]:
import pickle
import numpy as np
from collections import Counter
import pprint
import copy

In [2]:
with open('AMM_A350.pkl', 'rb') as f:
    data = pickle.load(f)
    
#sentences = data['PARAfull'][:20]
#data
# 1226552 rows

data = data[data['POStxt'].notna()]

In [3]:
data["Count"]=data.groupby("POStxt")["POStxt"].transform('count')
data_reduced = data.drop_duplicates(['POStxt'])
sentences = list(data_reduced['POStxt'])
count = list(data_reduced['Count'])

In [4]:
len(sentences)

226363

In [79]:
sentences = list(data_reduced['POStxt'])
counter = Counter([])
for i in range(len(sentences)):
        sentences[i] = sentences[i] + ' <eos>'
        counter += Counter(sentences[i].lower().split())
        
# Or use this command:
#with open('sentences_with_eos.pkl', 'rb') as f:
#    sentences, counter = pickle.load(f)

In [86]:
# Count number of keys with more than n appearances
n = 1
print( sum(counter[a] > n for a in counter.keys()) )

# Count proportion of words that we eliminate if we remove the one-time appearance words
print(sum(counter[a] == 1 for a in counter.keys()) / num_of_words)

7274
0.0


In [6]:
vocab = sorted(counter.keys())
vocab_size = len(vocab)
print('There are %d unique words in our dataset.' % vocab_size)
num_of_words = sum(counter.values())#sum( [counter[k] for k in counter.keys()] )
print('There are %d total words in our dataset.' % num_of_words)

There are 18890 unique words in our dataset.
There are 3380789 total words in our dataset.


In [80]:
# Count number of removed words (unique and with repetitions) with less than n appearances
n = 5

removed_unique_words = 0
removed_total_words = 0
for a in counter.keys():
    if counter[a] <= n:
        removed_unique_words +=1
        removed_total_words += counter[a]

print('There are %d words appearing less than %d times.' %(removed_unique_words,n))
print('It represents %d words we eliminate, i.e., %.2f%%  of all words.' %(removed_total_words, (removed_total_words/num_of_words) * 100 ))

There are 11617 words appearing less than 5 times.
It represents 22510 words we eliminate, i.e., 0.67%  of all words.


In [81]:
def preprocess_unk_words(sentences,counter,n):
    new_counter = counter
    for i in range(len(sentences)):
            sent_list = sentences[i].lower().split()
            for j in range(len(sent_list)):
                if counter[sent_list[j]] <= n:
                    new_counter.pop(sent_list[j], None)
                    sent_list[j] = '<unk>'
                    new_counter['<unk>'] += 1
                    sentences[i] = ' '.join(sent_list)
                    sentences[i] = sentences[i][0].upper() + sentences[i][1:]
    return (sentences, new_counter)

In [82]:
sentences, counter = preprocess_unk_words(sentences,counter,5)

In [83]:
vocab = sorted(counter.keys())
vocab_size = len(vocab)
print('There are %d unique words in our dataset.' % vocab_size)
num_of_words = sum(counter.values())#sum( [counter[k] for k in counter.keys()] )
print('There are %d total words in our dataset.' % num_of_words)

There are 7274 unique words in our dataset.
There are 3380789 total words in our dataset.


In [85]:
with open('sentences_with_eos_and_unk.pkl', 'wb') as handle:
    pickle.dump((sentences,counter), handle, protocol=pickle.HIGHEST_PROTOCOL)